In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, roc_auc_score, f1_score
from tqdm import tqdm
import json

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.options.mode.chained_assignment = None
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
ogrv = pd.read_csv('OGRV.csv', sep = ';')
ogrv['date'] = ogrv['date'].to_numpy().astype('datetime64[M]')

n_illness_days_df = (
    ogrv[ogrv['graphic_rule_level_1'] == 'Больничный']
    .groupby(['hash_tab_num', 'date'])
    .agg(n_illness_days=('work_shift_type', len))
    .reset_index()
)

n_illness_days_df.head(2)

,hash_tab_num,date,n_illness_days
0,0,2016-03-01,3
1,0,2016-04-01,10


In [3]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object 
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp)

In [4]:
# Считывание данных

sot = pd.read_csv('sotrudniki.csv', sep = ';')
sot['date'] = pd.to_datetime(sot['date'], format='%Y-%m-%d')

train_target_df = sot[['hash_tab_num', 'date', 'sick']]
train_target_df.head()

,hash_tab_num,date,sick
0,0,2015-04-01,0
1,0,2015-05-01,0
2,0,2015-06-01,0
3,0,2015-07-01,0
4,0,2015-08-01,0


---

In [5]:
sot_data = sot[[
    'hash_tab_num','date','category','gender','razryad_fact','work_experience_company',
    'name_fact_lvl5','education','home_to_work_distance'
]]

sot_data['gender'] = sot_data['gender'].map(lambda x: 1 if x == 'мужской' else 0)

In [6]:
# Создание вспомогательно датасета с информацией о количестве сотрудников в подразделении
# по фактическому месту работы

division_count = sot_data[['hash_tab_num','date','name_fact_lvl5']].\
groupby(['name_fact_lvl5','date']).agg('count').reset_index()

division_count.columns = ['name_fact_lvl5', 'date', 'personel_num']

sot_data = pd.merge(sot_data, division_count, how = 'left', on = ['date','name_fact_lvl5'])

In [7]:
# Создание dummy переменных

sot_data.education = sot_data['education']\
.map(lambda x: 'Высшее' if x in ['Высшее образование','Высшее-бакалавриат','Высшее-специалитет'] else(\
'Среднее_профессинальное' if x in ['Ср.профессиональное','Нач.профессиональное'] else 'Начальное_среднее'))

sot_data = pd.get_dummies(sot_data, columns = ['category','education','razryad_fact']).drop('name_fact_lvl5', axis = 1)
sot_data['orig_date'] = sot_data['date'].copy()

In [8]:
sot_data.head()

,hash_tab_num,date,gender,work_experience_company,home_to_work_distance,personel_num,category_Рабочие,category_Руководители,category_Служащие,category_Специалисты,education_Высшее,education_Начальное_среднее,education_Среднее_профессинальное,razryad_fact_0,razryad_fact_1,razryad_fact_2,razryad_fact_3,razryad_fact_4,razryad_fact_5,razryad_fact_6,orig_date
0,0,2015-04-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,2015-04-01
1,0,2015-05-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,2015-05-01
2,0,2015-06-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,2015-06-01
3,0,2015-07-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,2015-07-01
4,0,2015-08-01,1,9.0,NaN,NaN,1,0,0,0,0,1,0,0,0,0,1,0,0,0,2015-08-01


In [9]:
submission_extra = sot_data[sot_data['orig_date'] == pd.to_datetime('2019-08-01')]
submission_extra['target'] = 0

---

In [10]:
with open('transformed_data/date_of_birth.json', 'r') as f:
    date_of_birth_dict = json.load(f)
    date_of_birth_dict = {int(k): int(v) for k, v in date_of_birth_dict.items()}


def calc_age(hash_tab_num, calc_date, date_of_birth_dict):
    cur_date = int(calc_date)
    birth_date = date_of_birth_dict[hash_tab_num]
    age = cur_date - birth_date
    return age


with open('transformed_data/relatives_info.json', 'r') as f:
    relatives_dict = json.load(f)
    relatives_dict = {int(k): v for k, v in relatives_dict.items()}


def calc_relatives_bins(hash_tab_num, calc_date, relatives_dict):
    '''
    bins:
        0: 0 - 3: младенец
        1: 4 - 7: ребенок
        2: 8 - 18: школьник
        3: 19 - 35: молодежь :)
        4: 36 - 55(F), 60(M): предпенсионный возраст
        5: 55(F), 60(M) - +++: пенсионер
        6: кол-во родственников мужского рода
        7: кол-во родственников женского рода
    '''
    
    bins = [0] * 8
    if hash_tab_num not in relatives_dict:
        return bins
    
    cur_date = int(calc_date)
    for (sex, birth_date) in relatives_dict[hash_tab_num]:
        if sex == 'M':
            bins[6] += 1
        elif sex == 'F':
            bins[7] += 1
            
        if birth_date < 0:
            continue
            
        age = cur_date - birth_date
        if age < 0:
            continue
        elif age <= 3:
            bins[0] += 1
        elif age <= 7:
            bins[1] += 1
        elif age <= 18:
            bins[2] += 1
        elif age <= 35:
            bins[3] += 1
        else:
            if (sex == 'M' and age >= 60) or (sex == 'F' and age >= 55):
                bins[5] += 1
            else:
                bins[4] += 1
    return bins

In [11]:
def target_date_features(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['age'] = df.apply(lambda x: calc_age(x['hash_tab_num'], x['year'], date_of_birth_dict), axis=1)
    df['is_pensioner'] = (((df['age'] >= 60) & (df['gender'] == 1)) | ((df['age'] >= 55) & (df['gender'] == 0))).astype(int)
    df['relatives'] = df.apply(lambda x: calc_relatives_bins(x['hash_tab_num'], x['year'], relatives_dict), axis=1)
    for i in range(8):
        df[f'relatives_{i}'] = df['relatives'].apply(lambda x: x[i])
    df = df.drop(columns=['year', 'relatives'])
    
    return df

## Add previous ilnesses

In [12]:
print(sot_data.shape)
new_start_date = to_datetime(sot_data['orig_date'].min()) + relativedelta(months=6)
sot_data = sot_data[sot_data['orig_date'] >= new_start_date]
print(sot_data.shape)

(99214, 21)
(93221, 21)


In [13]:
def add_illness_days(df):
    for month_shift in [1, 2, 3, 4, 5, 6]:
        df['date'] = df['orig_date'] - pd.DateOffset(months=month_shift)
        df = pd.merge(df, n_illness_days_df, on=['hash_tab_num', 'date'], how='left')
        df['n_illness_days'] = df['n_illness_days'].fillna(-1).astype('int')
        df = df.rename(columns={'n_illness_days': f'n_illness_days_{month_shift}'})
    return df

In [14]:
sot_data = add_illness_days(sot_data)
submission_extra = add_illness_days(submission_extra)

In [16]:
results = []
models = []

for months in range(1, 13):
    sot_data['date'] = sot_data['orig_date'] + pd.DateOffset(months=months)
    df_train = pd.merge(sot_data, train_target_df, on=['hash_tab_num', 'date'])
    df_train = target_date_features(df_train)

    X = df_train.drop(columns=['hash_tab_num', 'date', 'orig_date', 'sick']).fillna(-100)
    y = df_train['sick']
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y,
    )
    
    model = RandomForestClassifier(n_estimators=200)
    model.fit(X_train, y_train)
    models.append(model)

    p, r, thresholds = precision_recall_curve(y_test, model.predict_proba(X_test)[:, 1])
    f1_scores = 2 * r * p / (r + p)
    f1_scores = f1_scores[p > 0]
    th = thresholds[np.argmax(f1_scores)]
    print(months, th, max(f1_scores))

    submission_extra['date'] = submission_extra['orig_date'] + pd.DateOffset(months=months)
    df_test = target_date_features(submission_extra)
    
    df_test = df_test.drop(columns=['hash_tab_num', 'date', 'orig_date', 'target']).fillna(-100)
    submission_extra['target'] = (model.predict_proba(df_test)[:, 1] >= th).astype(int)
    results.append(submission_extra[['hash_tab_num', 'date', 'target']].copy())
    
result_df = pd.concat(results, ignore_index=True)
result_df.head()

1 0.19 0.323364202216487
2 0.18 0.3293946024799416
3 0.19 0.3253146407519516
4 0.21 0.32367849384503977
5 0.18 0.328525641025641
6 0.21833333333333332 0.33164403000202713
7 0.19 0.32204907755686907
8 0.185 0.3224043715846995
9 0.18 0.33382461311717027
10 0.18 0.3269598470363289
11 0.185 0.332604808265448
12 0.23 0.3374233128834355


,hash_tab_num,date,target
0,0,2019-09-01,1
1,1,2019-09-01,0
2,2,2019-09-01,0
3,3,2019-09-01,1
4,4,2019-09-01,0


In [22]:
sorted(zip(models[0].feature_importances_, X_train))

[(3.9293222373502877e-05, 'category_Служащие'),
 (0.0002462157230397062, 'razryad_fact_1'),
 (0.0005464754039567421, 'category_Специалисты'),
 (0.0008090219452134489, 'category_Руководители'),
 (0.0017127778442173796, 'razryad_fact_6'),
 (0.001742901758003719, 'category_Рабочие'),
 (0.002805546320707022, 'is_pensioner'),
 (0.003227443847889691, 'razryad_fact_0'),
 (0.003963376005101811, 'education_Высшее'),
 (0.006602067686274391, 'razryad_fact_5'),
 (0.007019434914386372, 'razryad_fact_2'),
 (0.009136071671756953, 'education_Начальное_среднее'),
 (0.009419241119991553, 'gender'),
 (0.009823888563966734, 'education_Среднее_профессинальное'),
 (0.009935706532837749, 'razryad_fact_4'),
 (0.010048185610300995, 'razryad_fact_3'),
 (0.010829461436609516, 'relatives_5'),
 (0.013619709975229467, 'relatives_0'),
 (0.01362717319443656, 'relatives_1'),
 (0.01828834717499448, 'relatives_4'),
 (0.020625990311766484, 'relatives_2'),
 (0.021381950164929105, 'relatives_3'),
 (0.027962758531691577, 'n

In [23]:
sorted(zip(models[-1].feature_importances_, X_train))

[(2.792951461429063e-05, 'category_Служащие'),
 (0.0003192472968838628, 'razryad_fact_1'),
 (0.0005744031798352845, 'category_Специалисты'),
 (0.0010356968618346616, 'category_Руководители'),
 (0.0017549361567989648, 'category_Рабочие'),
 (0.0018565759917234725, 'razryad_fact_6'),
 (0.0032328661022218053, 'is_pensioner'),
 (0.0032434505354505426, 'razryad_fact_0'),
 (0.003917243101740642, 'education_Высшее'),
 (0.007058844048286317, 'razryad_fact_5'),
 (0.007642287360506522, 'razryad_fact_2'),
 (0.008488808400501556, 'education_Начальное_среднее'),
 (0.008966422386197027, 'education_Среднее_профессинальное'),
 (0.0097768938720705, 'razryad_fact_3'),
 (0.009834809985117736, 'gender'),
 (0.010327513725400429, 'razryad_fact_4'),
 (0.011589537105387833, 'relatives_5'),
 (0.013493086890885337, 'relatives_0'),
 (0.013959955278446332, 'relatives_1'),
 (0.018628030083885853, 'relatives_4'),
 (0.0218437020020996, 'relatives_3'),
 (0.021871741226295587, 'relatives_2'),
 (0.025274173101300142, 'n

In [24]:
check_df = pd.read_csv('submission_check.csv', sep=';').drop(columns=['target'])
check_df['date'] = pd.to_datetime(check_df['date'], format='%Y-%m-%d')
check_df.head()

,hash_tab_num,date
0,0,2019-09-01
1,0,2019-10-01
2,0,2019-11-01
3,0,2019-12-01
4,0,2020-01-01


In [25]:
result_df_new = pd.merge(result_df, check_df, on=['hash_tab_num', 'date']).sort_values(['hash_tab_num', 'date'])
result_df_new.head()

,hash_tab_num,date,target
0,0,2019-09-01,1
1757,0,2019-10-01,1
3509,0,2019-11-01,1
5265,0,2019-12-01,1
7025,0,2020-01-01,1


In [26]:
result_df_new.to_csv('submission_10.csv', sep=';', index=False)